<a href="https://colab.research.google.com/github/JhonnyLimachi/Sigmoidal/blob/main/30_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 4.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*

---

# Feature Engineering

***Feature Engineering*** é usar dados, dentro do domínio do conhecimento, e criar *features* novas para o seu conjunto de dados. Quando você entende esse conceito e aplica corretamente as técnicas de engenharia, melhora ainda mais a capacidade de previsão do seu modelo de machine learning.


<center><img src="https://images.unsplash.com/photo-1504328345606-18bbc8c9d7d1?ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&ixlib=rb-1.2.1&auto=format&fit=crop&w=1050&q=80"width="70%"></center>


> *The algorithms we used are very standard for Kagglers. […]  We spent most of our efforts in feature engineering  (Xavier Conort)*


A verdade é que a engenharia de features demanda um conhecimento a respeito do seu problema. Por isso você deve investigar quais variáveis são importantes para o escopo identificado.

Vamos ver alguns exemplos de como criar novas features.

## Exemplo 1 - Mercado imobiliário de São Paulo

Normalmente, você vai pegar datasets que irão conter variáveis primárias, como é o caso dos dados deste exemplo.

Obviamente, poderíamos analisar e criar um modelo de machine learning para prever o preço de venda de um apartamento, apenas usando os dados originais que foram fornecidos.

No entanto, mesmo um bom modelo não irá conseguir capturar todas as nuances. Mesmo aquelas que seriam teoricamente simples, podem contribuir para melhorar o desempenho do seu algoritmo.

Os dados abaixo foram baixados diretamente [deste site](https://www.kaggle.com/argonalyst/sao-paulo-real-estate-sale-rent-april-2019), e foram disponibilizados publicamente por uma startup do setor imobiliário.

In [1]:
# importar os dados necessários
import pandas as pd
import numpy as np
import seaborn as sns
import geopy.distance

# importar dados para o dataframe
DATA_PATH = "https://raw.githubusercontent.com/carlosfab/dsnp2/master/datasets/sao-paulo-properties-april-2019.csv"
df = pd.read_csv(DATA_PATH)

# ver as 5 primeiras entradas
df.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
0,930,220,47,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.543138,-46.479486
1,1000,148,45,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.550239,-46.480718
2,1000,100,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.542818,-46.485665
3,1000,200,48,2,2,1,1,0,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.547171,-46.483014
4,1300,410,55,2,2,1,1,1,0,0,0,Artur Alvim/São Paulo,rent,apartment,-23.525025,-46.482436


Primeiramente, vou extrair apenas os apartamentos que foram colocados à venda. Como identifiquei que a coluna `Negotiation Type` contém apenas os valores `rent` e `sale`, basta fazer `'Negotiation Type'] == 'sale'`.

In [2]:
df_venda = df[df['Negotiation Type'] == 'sale'].copy()
df_venda.head()

,Price,Condo,Size,Rooms,Toilets,Suites,Parking,Elevator,Furnished,Swimming Pool,New,District,Negotiation Type,Property Type,Latitude,Longitude
4901,732600,1000,74,1,2,1,2,1,0,1,0,Vila Madalena/São Paulo,sale,apartment,-23.552129,-46.692244
4902,1990000,2400,164,4,5,2,3,1,1,1,0,Vila Madalena/São Paulo,sale,apartment,-23.551613,-46.699106
4903,720000,700,70,2,2,1,1,1,0,1,1,Vila Madalena/São Paulo,sale,apartment,-23.547687,-46.692594
4904,1680000,1580,155,3,5,3,2,1,0,1,0,Vila Madalena/São Paulo,sale,apartment,-23.552590,-46.691104
4905,1200000,900,56,2,2,1,2,0,1,1,0,Vila Madalena/São Paulo,sale,apartment,-23.553087,-46.697890


Uma métrica importante, que é levada em consideração na decisão de compra de imóvel, é o valor do metro quadrado.

Para conseguir esse número, basta dividir `Price` por `Size` e criar uma nova coluna.

In [ ]:
df_venda['price_per_size'] = (df_venda['Price'] / df_venda['Size']).astype(int)

df_venda.head()

Vamos imaginar que, especificamente para o seu projeto, a distância do imóvel para o Aeroporto de Guarulhos é uma característica relevante. Por que não criar uma coluna indicando essa distância, em relação a cada imóvel do dataset, e inserir essa variável no modelo?

Para calcular a distância entre duas coordenadas, vou usar o método `geopy.distance`